In [ ]:
!pip install langchain_groq langchain_core langchain_community

In [ ]:
from langchain_groq import ChatGroq


In [ ]:
!pip install pypdf

In [ ]:
!pip install chromadb

In [ ]:
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

In [ ]:
def initialize_llm():
  llm = ChatGroq(
    temperature = 0,
    groq_api_key = "--",
    model_name = "llama-3.3-70b-versatile"
)
  return llm

In [ ]:
from langchain_openai import ChatOpenAI

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "--"  

openai_llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini")

In [ ]:
def create_vector_db():
  loader = DirectoryLoader("/content/data/", glob = '*.pdf', loader_cls = PyPDFLoader)
  documents = loader.load()

  if not documents:
      print("No PDF documents found in /content//. Please upload your PDF files to this directory.")
      return None  

  text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 50)
  texts = text_splitter.split_documents(documents)
  embeddings = HuggingFaceBgeEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
  vector_db = Chroma.from_documents(texts, embeddings, persist_directory = './chroma_db')
  vector_db.persist()

  print("ChromaDB created and data saved")

  return vector_db

In [ ]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate

def setup_qa_chain(vector_db, llm):
    memory = ConversationBufferMemory(
        memory_key="chat_history",
        return_messages=True,
        output_key="result"
    )

    retriever = vector_db.as_retriever()

    prompt_template = """ your name is Luna(introduce yourself in the first interaction only),
    You are a compassionate mental health chatbot.
    Begin with a short intro to the topic. Answer the user's question in a caring way and don't talk alot and answer in bullet points (if needed).
    Chat History:{chat_history}
    Context: {context}
    human: {question}
    mental health chatbot:"""
    qa_prompt = PromptTemplate(
        template=prompt_template,
        input_variables=["context", "question"]
    )

    
    qa_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=retriever,
        memory=memory,
        combine_docs_chain_kwargs={"prompt": qa_prompt},
        return_source_documents=True,
        output_key="result"
    )

    return qa_chain

In [ ]:
def main():
  print("Intializing Chatbot.........")
  llm = initialize_llm()

  db_path = "/content"

  if not os.path.exists(db_path):
    vector_db  = create_vector_db()
    if vector_db is None: 
      print("Chatbot: Could not create vector database. Please check the /content/ directory.")
      return
  else:
    embeddings = HuggingFaceBgeEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
    vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)
  qa_chain = setup_qa_chain(vector_db, llm)

  while True:
    query = input("\nHuman: ")
    if query.lower()  == "exit":
      print("Chatbot: Take Care of yourself, Goodbye!")
      break
    response = qa_chain.invoke({"question": query}) 
    print(f"Chatbot: {response['result']}") 
    if 'source_documents' in response:
        print("Source Documents:")
        for doc in response['source_documents']:
            print(f"- {doc.metadata['source']}")

if __name__ == "__main__":
  main()

Intializing Chatbot.........

Human: hello i am abdo


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Chatbot: Hello Abdo, I'm Luna, a compassionate mental health chatbot. I'm here to listen and support you. Mental health is just as important as physical health, and it's great that you're taking the first step by reaching out. How can I assist you today?
Source Documents:

Human: I think I might harm myself


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Chatbot: Abdo, I'm so glad you reached out for support. Your safety is my top priority. Here are some important steps:
* I want you to know that you're not alone, and I'm here to listen without judgment.
* If you're in immediate danger, please call emergency services or a crisis hotline in your area.
* Can you tell me more about what's been going on and how you're feeling? Sometimes talking about it can help.
Source Documents:


In [ ]:
!pip install ragas

In [ ]:
pip install openai

In [ ]:
!pip install aidial-rag-eval

In [ ]:
from ragas.llms import LangchainLLMWrapper


def get_rag_response(question, qa_chain):
    response = qa_chain.invoke({"query": question})
    print(response)  # Debug
    return {
        "answer": response.get("result") or response.get("answer"),
        "contexts": [doc.page_content for doc in response.get("source_documents", [])]
    }
# FOR RAGAS EVALUATION
def run_ragas_evaluation():
    """Complete RAGAS evaluation function"""
    from ragas import evaluate
    from datasets import Dataset
    from ragas.metrics import faithfulness, answer_relevancy, answer_correctness, context_precision, context_recall

    # Initialize components
    llm = initialize_llm()

    db_path = '/content'
    if not os.path.exists(db_path):
        vector_db = create_vector_db()
        if vector_db is None:
            return
    else:
        embeddings = HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
        vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)

    qa_chain = setup_qa_chain(vector_db, llm)

    # Test data
    test_questions = [
        "What are common symptoms of depression?",
        "How can I manage anxiety?",
        "What is cognitive behavioral therapy?",
        "How to help someone with suicidal thoughts?",
        "What are healthy coping mechanisms for stress?"
    ]

    ground_truths = [
        "Common symptoms include persistent sadness, loss of interest, fatigue, changes in appetite, sleep disturbances, and difficulty concentrating.",
        "Anxiety can be managed through therapy, mindfulness practices, regular exercise, breathing techniques, and medication when prescribed by a professional.",
        "Cognitive Behavioral Therapy (CBT) is a short-term therapy that helps identify and change negative thought patterns and behaviors.",
        "Take all signs seriously, listen without judgment, remove means of self-harm, contact crisis services, and encourage professional help.",
        "Healthy coping includes exercise, meditation, social connection, time management, and seeking professional support."
    ]

    # Generate responses
    answers = []
    contexts = []

    for question in test_questions:
        rag_response = get_rag_response(question, qa_chain)
        answers.append(rag_response["answer"])
        contexts.append(rag_response["contexts"])

    # Create evaluation dataset
    evaluation_dataset = Dataset.from_dict({
        "question": test_questions,
        "answer": answers,
        "contexts": contexts,
        "ground_truth": ground_truths
    })
    ragas_llm = LangchainLLMWrapper(llm)
    # Run evaluation
    result = evaluate(
        evaluation_dataset,
        metrics=[faithfulness, answer_relevancy, answer_correctness, context_precision, context_recall],
        llm=ragas_llm
    )


    print("RAGAS Evaluation Results:")
    for metric_name, score in result.items():
        print(f"{metric_name}: {score:.4f}")

    return result

if __name__ == "__main__":

    run_ragas_evaluation()  

In [ ]:
!pip install gradio

In [ ]:
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_groq import ChatGroq
import os
import gradio as gr

# Initialize components outside the functions to maintain state
vector_db = None
qa_chain = None

def initialize_components():
    global vector_db, qa_chain

    # 1. Initialize LLM
    llm = ChatGroq(
        temperature=0,
        groq_api_key="--",
        model_name="llama-3.3-70b-versatile"
    )

    # 2. Set up vector database
    db_path = "./chroma_db"
    embeddings = HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

    if not os.path.exists(db_path):
        try:
            # Load PDFs and create new database
            loader = DirectoryLoader("/content/", glob='*.pdf', loader_cls=PyPDFLoader)
            documents = loader.load()

            if not documents:
                raise Exception("No PDF documents found in /content/. Please upload PDF files.")

            text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
            texts = text_splitter.split_documents(documents)
            vector_db = Chroma.from_documents(texts, embeddings, persist_directory=db_path)
            vector_db.persist()
            print("Database created and data saved")
        except Exception as e:
            print(f"Error creating database: {str(e)}")
            vector_db = None
    else:
        # Load existing database
        try:
            vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)
        except Exception as e:
            print(f"Error loading database: {str(e)}")
            vector_db = None

    # 3. Set up QA chain if database is available
    if vector_db is not None:
        retriever = vector_db.as_retriever()
        prompt_template = """You are a compassionate mental health chatbot. Respond thoughtfully to the following question:
        {context}
        User: {question}
        Chatbot:"""
        PROMPT = PromptTemplate(template=prompt_template, input_variables=['context', 'question'])
        qa_chain = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type="stuff",
            retriever=retriever,
            chain_type_kwargs={"prompt": PROMPT},
            return_source_documents=True
        )
    else:
        qa_chain = None

# Initialize components when the script runs
print("Initializing chatbot components...")
initialize_components()

def chatbot_response(message, history):
    if not message.strip():
        return "Please enter a valid message"

    if qa_chain is None:
        return "Chatbot is not ready. Database might not be loaded properly."

    try:
        response = qa_chain({"query": message})
        return response["result"]
    except Exception as e:
        return f"Error processing your request: {str(e)}"

# Create Gradio interface
with gr.Blocks(theme=gr.themes.Soft()) as app:
    gr.Markdown("""
    # 🤖 Mental Health Support Chatbot
    A compassionate chatbot for mental health support.
    For serious concerns, please contact a professional.
    """)

    chatbot = gr.ChatInterface(
        fn=chatbot_response,
        examples=["I feel sad today", "How to manage anxiety?", "I'm feeling overwhelmed"],
        title="Mental Health Support"
    )

    gr.Markdown("""
    Note: This chatbot provides general support only.
    In case of emergency, please contact local health services.
    """)

# Launch the app
if __name__ == "__main__":
    app.launch(share=True)

Initializing chatbot components...


/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:345: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(
Exception in thread Thread-22 (run):
Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/server.py", line 67, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 26, in run
    loop = asyncio.get_event_loop()
           ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 40, in _get_event_

KeyboardInterrupt: 